In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Create levels with multiple starting conformations

In [2]:
import copy

min_rmsd = 0
max_rmsd = 1
stop = False
level = 0
rmsds_dict = {}

while stop == False:
    stop = True
    for i in range(64):
        mol1 = copy.deepcopy(mol)
        mol1.dropFrames(keep=[i])


        mol1.align('name CA', refmol=ref_mol)
        rmsd_proj = MetricRmsd(ref_mol, 'name CA', centerstr='name CA', pbc=False)
        rmsd = rmsd_proj.project(mol1)

        if min_rmsd <= rmsd <= max_rmsd:
            rmsds_dict['level'+str(level + 1)] = rmsd.item()
            print(rmsd)
            min_rmsd += 1
            max_rmsd += 1
            stop = False       
            path = '/workspace7/torchmd-AD/train_curriculum/cln/levels/level' + str(level)
            #os.mkdir(path)
            mol1.write(os.path.join(path, 'chignolin_cln025.xtc' ))
            level += 1
    
    

NameError: name 'mol' is not defined

In [96]:
import pandas as pd
pd.DataFrame.from_dict(rmsds_dict, orient='index', columns = ['starting rmsd'])

,starting rmsd
level1,0.873917
level2,1.463428
level3,2.858741
level4,3.779713
level5,4.009621
level6,5.704338
level7,6.318629


In [21]:
mol_curriculum.write('/workspace7/torchmd-AD/train_curriculum/cln/psf/chignolin_cln025.psf')
mol_curriculum.write('/workspace7/torchmd-AD/train_curriculum/cln/xtc/chignolin_cln025.xtc')

NameError: name 'mol_curriculum' is not defined

## Create levels with multiple starting conformations

In [3]:
from moleculekit.molecule import Molecule
import os
import importlib
from torchmdexp.losses.rmsd import rmsd
from torchmdexp.losses.tmscore import tm_score
from torchmdexp.samplers.utils import get_native_coords, get_embeddings
from moleculekit.projections.metricrmsd import MetricRmsd
from moleculekit.projections.metrictmscore import MetricTMscore
import numpy as np
import torch

In [70]:
# Reference Molecule
prot = 'lambda_ca'
name = 'lambda'

mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca_top_dih.psf')
ref_mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca.pdb')


# Read the array with trajectory data

mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca_top_dih.psf')
arr = np.load(f'/workspace2/fast_folders_cgnet/multiprotein/data_dih/{prot}_coords_stride10.npy')[:1000000]
mol.coords = np.moveaxis(arr, 0, -1)

mol.filter('name CA')

array([], dtype=int32)

In [71]:
# Compute the rmsd of each frame against the reference structure
mol.align('name CA', refmol=ref_mol)
rmsd_proj = MetricRmsd(ref_mol, 'name CA', centerstr='name CA', pbc=False)
rmsd = rmsd_proj.project(mol)

In [72]:
mol.coords.shape
#arr.shape

(80, 3, 1000000)

In [73]:
# Sample frames for different levels and save them in levels folders
from statistics import mean
n_levels = int(1 // 1)
min_rmsd = 0
max_rmsd = 3

path = '/workspace7/torchmd-AD/train_curriculum/fastfolders_contacts/level_0/init_states/'
mol_frames = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca_top_dih.psf')

for n in range(n_levels):
    
    indexes = np.where(np.logical_and(rmsd>=min_rmsd, rmsd<=max_rmsd))[0]
    frames = arr[indexes, :, :][:200]
    
    print(mean(rmsd[indexes]))
    print(frames.shape)
    
    directory = path
    #os.mkdir(directory)
    
    mol_frames.coords = np.moveaxis(frames, 0, -1)
    mol_frames.write(path + f'{prot}.xtc')
    
    #np.save(os.path.join(directory, 'ww_ca.npy'), frames)
    #min_rmsd += 2
    #max_rmsd += 2
    
    

2.573829
(200, 80, 3)


# Add folded conformations

In [89]:
from moleculekit.molecule import Molecule
import os
import importlib
from torchmdexp.losses.rmsd import rmsd
from torchmdexp.losses.tmscore import tm_score
from torchmdexp.samplers.utils import get_native_coords, get_embeddings
from moleculekit.projections.metricrmsd import MetricRmsd
from moleculekit.projections.metrictmscore import MetricTMscore
import numpy as np
import torch

device = "cuda:0"
precision = torch.float

In [86]:
prot = 'lambda_ca'
name = 'lambda'
level = 0
num = 32
mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca_top_dih.psf')
mol.read(f'/workspace7/torchmd-AD/train_curriculum/fastfolders_contacts/level_0/init_states/{prot}.xtc')
ref_mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca.pdb')

In [47]:
for i in range(mol.coords.shape[2]):
    mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca_top_dih.psf')
    mol.read(f'/workspace7/torchmd-AD/train_curriculum/fastfolders_contacts/level_0/init_states/{prot}.xtc')
    #mol.read(f'/shared/carles/torchmd-exp/data/{name}/{name}_{num}_unfolded_coords.xtc')
    ref_mol = Molecule(f'/shared/carles/torchmd-exp/data/{name}/{name}_ca.pdb') 
    
    mol.coords = np.expand_dims(mol.coords[:, : , i], axis=-1)
    mol.box = np.expand_dims(mol.box[:, 0], axis=-1)


    native_coords = get_native_coords(ref_mol, device)
    native_coords = native_coords.type(precision)
    coords = get_native_coords(mol, device)
    coords = coords.type(precision)


    print('Starting rmsd: ', rmsd(coords, native_coords).item(), f'index {i}')#, 'Starting log_rmsd: ', log_rmsd(coords, native_coords))
    #print('Starting tmscore: ', tm_score(coords, native_coords), 'Starting l2_rmsd: ', l2_rmsd(coords, native_coords))

Starting rmsd:  1.9103952646255493 index 0
Starting rmsd:  1.9679701328277588 index 1
Starting rmsd:  1.8667999505996704 index 2
Starting rmsd:  1.8043235540390015 index 3
Starting rmsd:  1.8404288291931152 index 4
Starting rmsd:  1.9307560920715332 index 5
Starting rmsd:  0.9256690740585327 index 6
Starting rmsd:  1.1236501932144165 index 7
Starting rmsd:  1.2362557649612427 index 8
Starting rmsd:  0.8981114029884338 index 9
Starting rmsd:  1.243046760559082 index 10
Starting rmsd:  0.855055034160614 index 11
Starting rmsd:  0.7907766699790955 index 12
Starting rmsd:  1.0606472492218018 index 13
Starting rmsd:  1.1854370832443237 index 14
Starting rmsd:  1.3230206966400146 index 15
Starting rmsd:  0.8943288922309875 index 16
Starting rmsd:  1.140515685081482 index 17
Starting rmsd:  0.6264716982841492 index 18
Starting rmsd:  1.0466814041137695 index 19
Starting rmsd:  0.7556281089782715 index 20
Starting rmsd:  1.0684744119644165 index 21
Starting rmsd:  1.121641993522644 index 22
St

IndexError: index 200 is out of bounds for axis 2 with size 200

In [87]:
num_folded = mol.coords.shape[2]
for i in range(num_folded):
    mol.coords = np.append(mol.coords, ref_mol.coords, axis=-1)

In [88]:
mol.write(f'/workspace7/torchmd-AD/train_curriculum/fastfolders_half_native/level_0/init_states/{prot}.xtc')